In [1]:
import numpy as np
import pandas as pd

import pandas_datareader.data as web
from pandas_datareader import wb

import dask
import dask.dataframe as dd

import datetime

import requests
from zipfile import ZipFile

import os

In [2]:
def worldBandData(indicator, endDate, col_name):
    """
    
    """
    df = wb.download(country='All', indicator=indicator, start=1960, end=endDate)
    df.columns = ['GDP_growth_annual_perc']
    return df

In [3]:
def stooqData(indicator):
    """
    
    """
    df = web.DataReader(indicator, 'stooq')
    
    return df

In [4]:
def loadDataSouces():
    df = pd.read_csv('../data/raw/dataSource.csv')
    return df

In [5]:
def eurostatData(indicator, start, end, retry_count):
    df = web.DataReader(indicator, 'eurostat', start=start, end=end, retry_count=retry_count)
    
    return d

In [6]:
def getFileFromStooq(url):
    """
    
    """
    r = requests.get(url, allow_redirects=True)
    if url.find('/'):
        filename = url.rsplit('/')[-1]
    open('../data/raw/{}'.format(filename), 'wb').write(r.content)
    if filename.endswith('.zip'):
        z = ZipFile('../data/raw/{}'.format(filename))
        z.extractall('../data/raw')

In [10]:
getFileFromStooq('https://static.stooq.com/db/h/d_world_txt.zip')

trying extract


In [7]:
def saveData(df, source, indicator):
    df.to_parquet('../data/raw/{0}_{1}.parquet'.format(source, indicator))

In [5]:
dataSources = loadDataSouces()

In [8]:
file_dirs = []
for root, dirs, files in os.walk('../data/raw/data', topdown=True):
    if files:
        for f in files:
            file_dirs.append(os.path.join(root, f))

In [9]:
data_dirs = pd.DataFrame(file_dirs, columns=['location'])

In [10]:
data_dirs['instrument_type'] = data_dirs.location.str.split('/').str.get(6)

In [11]:
data_dirs['instrument'] = data_dirs.location.str.split('/').str.get(7).str.replace('.txt', '')

In [12]:
data_dirs.instrument_type.value_counts()

currencies\other        1928
commodities              530
commodities cash         229
cryptocurrencies         110
money market              81
currencies\major          66
indices                   61
bonds                     32
lme                       18
stooq stocks indices      13
Name: instrument_type, dtype: int64

In [13]:
data_dirs.head()

,location,instrument_type,instrument
0,../data/raw/data/daily/world/lme/na_c.f.txt,lme,na_c.f
1,../data/raw/data/daily/world/lme/ah_3.f.txt,lme,ah_3.f
2,../data/raw/data/daily/world/lme/zs_3.f.txt,lme,zs_3.f
3,../data/raw/data/daily/world/lme/ni_c.f.txt,lme,ni_c.f
4,../data/raw/data/daily/world/lme/sn_c.f.txt,lme,sn_c.f


In [14]:
data_dirs.loc[0, 'parquet_location'] = '../data/processed/stooq/df.parquet'

In [15]:
for index, row in data_dirs.iterrows():
    df = pd.read_csv(row['location'], parse_dates=['<DATE>'])
    df.columns = [x.replace('<', '').replace('>', '') for x in df.columns.values]
    df.to_parquet('../data/processed/stooq/{0}.parquet'.format(row['instrument']), index=None)
    data_dirs.loc[index, 'parquet_location'] = '../data/processed/stooq/{0}.parquet'.format(row['instrument'])

In [17]:
data_dirs.to_parquet('../data/processed/data_dirs.parquet', index=None)